In [ ]:
# web scraping to pdf

In [13]:
# ft articles data [heading, link, date]

import requests
from bs4 import BeautifulSoup

news_data = []
keyword = 'exxon'
            

for page in range(1, 2):
    url = f"https://www.ft.com/search?q={keyword}&page={page}&sort=date&isFirstView=true"
    result = requests.get(url)
    result_content = result.content
    soup = BeautifulSoup(result_content, "lxml")
    
    for article in soup.findAll("div", {"class": "o-teaser"}):
        heading = article.find("div", {"class": "o-teaser__heading"}).get_text(strip=True) #find(text=True), find text is outdated
        link = article.find("div", {"class": "o-teaser__heading"}).find('a', href=True)
        date = article.find('time', {"class": "o-teaser__timestamp-date"})

        if heading and link and date:
            news_data.append([heading, link['href'], date.text.strip()])

print(f"Total articles collected in news_data: {len(news_data)}")

Total articles collected in news_data: 25


In [14]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from fpdf import FPDF
import uuid
import unidecode

path = './data3/'
bs4_strainer = bs4.SoupStrainer(class_=("o-topper__headline o-topper__headline--large", "n-content-body js-article__content-body"))
pdfs_counter = 0

for item in news_data:
    myurl = 'https://www.ft.com' + item[1]
    
    loader = WebBaseLoader(
        web_paths=(myurl,),
        bs_kwargs={"parse_only": bs4_strainer},
    )
    docs = loader.load()

    content = docs[0].page_content
    
    heading1 = unidecode.unidecode(item[0])
    date1 = unidecode.unidecode(item[2])
    content = unidecode.unidecode(content)

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Title: " + heading1, ln=True)
    pdf.cell(200, 10, txt="Date: " + date1, ln=True)
    pdf.multi_cell(0, 10, txt=content)  # Use multi_cell for content to handle line breaks

    # Generate a unique file name
    file_name = str(uuid.uuid4()) + ".pdf"
    full_path = path + file_name

    # Output PDF for this article
    pdf.output(full_path)
    pdfs_counter += 1
    
print(f"{pdfs_counter} files saved to data3 folder.")


25 files saved to data3 folder.
